## Spooky author identification challenge

### This challenge invites kagglers to identify the horror story author from the given text snippets. The challenge is little different from other nlp problems because we need to find the signature of the author from his writing style than simply understanding the context vectors. Hence word vectors may be of little help here. I am using keras embeddings instead of word2vec and combine features from LSTM and CNN(to be able to find pattern translation) and then pass the combined features to another hidden layer in the neural network. 

In [ ]:
import pandas as pd
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from keras.layers.core import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import EarlyStopping
from keras import optimizers

In [ ]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [ ]:
from keras.utils import np_utils
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [ ]:
vocab_size = 10000
xtrain_enc = [one_hot(d, vocab_size) for d in xtrain]
xvalid_enc = [one_hot(d, vocab_size) for d in xvalid]

In [ ]:
max_length = 300
padded_docs_train = pad_sequences(xtrain_enc, maxlen=max_length, padding='post')
padded_docs_valid = pad_sequences(xvalid_enc, maxlen=max_length, padding='post')